<img src="https://raw.githubusercontent.com/UXARRAY/uxarray/main/docs/_static/images/logos/uxarray_logo_h_dark.svg"
     width="30%"
     alt="UXarray logo"
     align="right"
/>

# Spatial Selection Operators

### In this tutorial, you'll learn:

* Using UXarray to select specific regions from an unstructured grid


### Related Documentation

* [UXarray Subsetting Documentation](https://uxarray.readthedocs.io/en/latest/user-guide/subset.html)
* [UXarray Cross-Section Documentation](https://uxarray.readthedocs.io/en/latest/user-guide/cross-sections.html)

### Prerequisites

| Concepts                                      | Importance | Notes |
|-----------------------------------------------| --- | --- |
| [UXDataset & UxDataArray Notebook](uxds-uxda) | Necessary | |

**Time to learn**: 10 minutes

-----


## Overview

When working with unstructured grids, a region or zone rather than the entire global grid might be of interest for data analysis and visualization. Such spatial selection of the grid/data might help with not only analyzing/plotting a specific region/zone, but also reducing the data size and increasing the performance as well as allowing the entire plots to be effectively displayed on the screen.

This notebook showcases how to spatially subset a grid and take a cross-section of the grid at a constant latitude and longitude.

## Subsetting

UXarray provides functionality for subsetting the grid into a **bounding box**, **bounding circle**, or **K-nearest neighbors**. 

Before demonstrating them, let us load some global data first:

### Load Data

In [ ]:
# Import
import cartopy.crs as ccrs
import geoviews.feature as gf
import uxarray as ux

grid_path = "../../meshfiles/x1.655362.grid.nc"
data_path = "../../meshfiles/x1.655362.data.nc"

# Open dataset and grab a data variable of interest
uxds = ux.open_dataset(grid_path, data_path)
uxda = uxds["relhum_200hPa"][0]

### Plot The Global Data

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Note!</p>
    The visualizations throughout this tutorial are only for demonstrating the results of the subsetting and cross-sections versus the global grid. Since the details of plotting with UXarray will be covered in the next chapter, we will not go over any details of the plots here.
</div>

In [ ]:
plot_opts = {"width": 700, "height": 350}

features = gf.coastline(
    projection=ccrs.PlateCarree(), line_width=0.4, scale="50m"
) * gf.states(projection=ccrs.PlateCarree(), line_width=0.4, scale="50m")

clim = (uxda.values.min(), uxda.values.max())

uxda.plot(
    rasterize=True, periodic_elements="exclude", title="Global Grid", **plot_opts
) * features

### Generate Various Subsets

In [ ]:
# Bounding box around Boulder, CO
ref_lon = -105.2705
ref_lat = 40.0150
ref_offset = 1

lon_bounds = (ref_lon - ref_offset, ref_lon + ref_offset)
lat_bounds = (ref_lat - ref_offset, ref_lat + ref_offset)

# Subset the global data variable and its grid using the bounding box
uxda_bbox = uxda.subset.bounding_box(lon_bounds, lat_bounds)

In [ ]:
# Now use a bounding circle subsetting
ref_center = [ref_lon, ref_lat]

uxda_bcircle = uxda.subset.bounding_circle(ref_center, ref_offset)

In [ ]:
# Now use a K-nearest neighbor subsetting
k_nn = 60

uxda_nn = uxda.subset.nearest_neighbor(ref_center, k=k_nn, element="nodes")

### Plot The Subsets

In [ ]:
plot_bbox = (
    uxda_bbox.plot(
        rasterize=True,
        periodic_elements="exclude",
        clim=clim,
        title="Bounding Box Subset around Boulder, CO (Corner Node Query)",
        **plot_opts,
    )
    * features
)

plot_bcircle = (
    uxda_bcircle.plot(
        rasterize=True,
        periodic_elements="exclude",
        clim=clim,
        title="Bounding Circle Subset around Boulder, CO (Corner Node Query)",
        **plot_opts,
    )
    * features
)

plot_nn = (
    uxda_nn.plot(
        rasterize=True,
        periodic_elements="exclude",
        clim=clim,
        title="K-Nearest Neighbor Subset around Boulder, CO (Corner Node Query)",
        **plot_opts,
    )
    * features
)

(plot_bbox + plot_bcircle + plot_nn).cols(1)

## Cross-sections

Similarly, UXarray provides functionality for taking cross-sections of a grid/data at constant **latitudes** and **longitudes**. 

### Load Data

In [ ]:
# Data paths
grid_path = "../../meshfiles/outCSne30.grid.ug"
data_path = "../../meshfiles/outCSne30.data.nc"

# Open dataset and grab a data variable of interest
uxds = ux.open_dataset(grid_path, data_path)
uxda = uxds["psi"]

### Plot The Global Data

In [ ]:
projection = ccrs.Robinson()

features_2 = gf.coastline(
    projection=projection, line_width=0.4, scale="50m"
) * gf.states(projection=projection, line_width=0.4, scale="50m")

uxda.plot(
    cmap="inferno",
    periodic_elements="split",
    projection=projection,
    title="Global Plot",
) * features_2

### Generate Cross-Sections

In [ ]:
# Cross-section at the latitude of Boulder, CO
cross_lat = 40.0150

uxda_cross_lat = uxda.cross_section.constant_latitude(cross_lat)

In [ ]:
# Now cross-section at the longitude of Boulder, CO
cross_lon = -105.2705

uxda_cross_lon = uxda.cross_section.constant_longitude(cross_lon)

### Plot The Cross-Sections

In [ ]:
plot_cross_lat = (
    uxda_cross_lat.plot(
        rasterize=False,
        backend="bokeh",
        cmap="inferno",
        projection=projection,
        global_extent=True,
        coastline=True,
        title=f"Cross-section at ({cross_lat}) degrees latitude around Boulder, CO",
    )
    * features_2
)

plot_cross_lon = (
    uxda_cross_lon.plot(
        rasterize=False,
        backend="bokeh",
        cmap="inferno",
        projection=projection,
        global_extent=True,
        coastline=True,
        title=f"Cross-section at ({cross_lon}) degrees longitude around Boulder, CO",
    )
    * features_2
)

(plot_cross_lat + plot_cross_lon).cols(1)

## What is next?
With this section, we have wrapped up this chapter, move on to the next chapter!